<a href="https://colab.research.google.com/github/PoliUniLu/Logigram/blob/master/CORA_temporal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#@title Initialize Enviroment


! git clone https://github.com/ZuzanaSebb/cora.git cora_repo && cd cora_repo && git checkout 25dc4999a5896026000715773621a9d51c7d3c11
import sys
sys.path.append('/content/cora_repo')
#!cd cora_repo && git pull && cd ..
#print(sys.path)
#!mv /content/cora /content/cora_repo
#!ls cora_repo

Cloning into 'cora_repo'...
remote: Enumerating objects: 377, done.
remote: Counting objects: 100% (377/377), done.
remote: Compressing objects: 100% (249/249), done.
remote: Total 377 (delta 265), reused 233 (delta 126), pack-reused 0
Receiving objects: 100% (377/377), 91.18 KiB | 2.40 MiB/s, done.
Resolving deltas: 100% (265/265), done.
Note: checking out '25dc4999a5896026000715773621a9d51c7d3c11'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 25dc499 multi-output fix


In [3]:
#@title  Initialize Settings
%load_ext google.colab.data_table
import pandas as pd
import numpy as np
from ipywidgets import interact,fixed, HBox,VBox,Label, Layout
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import files
import io,string

import cora
global df
global OptimizationContext 
global out_cols
global input_cols
global case_col
global temp_cols
global temp_map
global preprocessed_data 
global PI
global irr_sums
global irr_systems
global pi_details
global sol_details
global pi_chart
global df_original
PIs = None
irr_sums=None   

#--------------------import_controls--------------------------

def import_controls(fname):
  out = widgets.Output()
  global df_original

  import_data_button = widgets.Button(
        value = False,
        description = 'Set csv separator',
   ) 
 
  separator = widgets.Text(
        value=',',
        disabled = not fname.endswith('csv')
    )
  if fname.endswith('xls') or fname.endswith('xlsx'):
        df_original = pd.read_excel(io.BytesIO(data))
        out.clear_output()
        with out:
          display("Excel file succesfully loaded!")
  
  if fname.endswith('csv'):
      a = HBox([separator,import_data_button])
      display(a)
      out.clear_output()     
  else:
        out.clear_output()
        with out:
          display('Please enter a csv or an excel file!')
      
  
  def import_data(b):
    global df_original
    df_original = pd.read_csv(io.BytesIO(data),sep=separator.value)
    out.clear_output()
    with out:
        display(df_original)
  
  import_data_button.on_click(import_data)
  display(out)

#--------------------input_controls--------------------------


def create_input_controls(df):

    def create_case_column_selector(df):
        case_col_options = [x for x in df.columns] + ['-None-']
        selector = widgets.Dropdown(
        options = case_col_options,
        value = '-None-',
        disabled = False,
        )
        return selector

    def create_input_column_selector(df):
        default_value = list(df.columns)[1:len(df.columns)-2]
        input_col_options = [x for x in df.columns] 
        selector = widgets.SelectMultiple(
        options = input_col_options,
        value = default_value,
        disabled = False,
        )
        return selector
    
    def create_temporal_columns_selector(df):
        #default_value = ['-None-']
        input_col_options =  [x for x in df.columns] 
        selector = widgets.SelectMultiple(
        options = input_col_options,
        value = (),
        disabled = False,
        )
        return selector

    def create_output_columns_selector(df):    
        default_value = list(df)[-1]
        out_col_options = [x for x in df.columns]
        selector = widgets.SelectMultiple(
        options = out_col_options,
        value = [default_value],
        disabled=False,
        )
        return selector
    
    def create_tempo_columns_analysis(df):
        all_selectors = []
        for col in temp_cols:
          selector = widgets.SelectMultiple(
              options = input_cols,
              description = col,
              disabled = False

          )
          all_selectors.append(selector)
          
        return all_selectors


    def create_output_columns_analysis(df):
        out_cols_map = dict()
        for i in out_cols:
           out_cols_map[i] = df[i].unique()
        all_selectors = []
        for key in out_cols_map:
           selector = widgets.SelectMultiple(
               options = out_cols_map[key],
               value = [1],
               description = key,
               disabled = False
           )
           all_selectors.append(selector)
        return all_selectors
    
    #bottons
    show_data_button = widgets.Button(
        value=False,
        description='Set data'
    )  

    show_parameters = widgets.Button(
        value=False,
        description='OK'
    )  
     

    label_layout = Layout(width='400px',height='30px')
    out_columns = create_output_columns_selector(df)
    case_column = create_case_column_selector(df)
    temporal_columns = create_temporal_columns_selector(df)
    input_columns = create_input_column_selector(df)

    CaseIDbox = VBox([Label('Case Column ID',layout=label_layout),
                    case_column])
    Outputbox = VBox([Label('Outputs',layout=label_layout),out_columns])
    Inputbox = VBox([Label('Inputs',layout=label_layout),input_columns])
    Temporalbox = VBox([Label('Auxiliary inputs',layout=label_layout),temporal_columns])
    resbox = HBox([CaseIDbox,Outputbox,Inputbox,Temporalbox])
    display(resbox)
    resbox2 = HBox([show_data_button])
    display(resbox2)


    # Output
    out = widgets.Output()
    out.clear_output(wait=True)
    display(out)

    def on_button_show_click(b):
        global out_cols
        global input_cols
        global case_col
        global temp_cols
        out_cols = [x for x in out_columns.value]
        input_cols = [x for x in input_columns.value]
        case_col = case_column.value if case_column.value\
                                    != '-None-' else None
        if len(temporal_columns.value) == 0:
          temp_cols = None
          if case_col in set(out_cols+input_cols):
            out.clear_output()
            with out:
              display('CaseID column cannot be part of\
                     input or output columns!')
            return()

        else:
          temp_cols = [x for x in temporal_columns.value]
          if case_col in set(out_cols+input_cols+temp_cols):
              out.clear_output()
              with out:
                display('CaseID column cannot be part of\
                     input or output columns!')
          if len(set(input_cols).intersection(set(temp_cols))) >0:
              out.clear_output()
              with out:
                display('Input and Auxilary columns have to be distinct!')
              return()
          if len(set(out_cols).intersection(set(temp_cols))) >0:
              out.clear_output()
              with out:
                display('Output and Auxilary columns have to be distinct!')
              return()
        
          
        if len(set(input_cols).intersection(set(out_cols))) > 0:
          out.clear_output()
          with out:
            display('Input and Output columns have to be distinct!')
          return()
       
        if case_col is None and temp_cols is None:
          res = df_original[input_cols+out_cols].copy()
        elif case_col is not None and temp_cols is None:
          res = df_original[[case_col]+input_cols+out_cols].copy()
        elif case_col is None and temp_cols is not None:
          res = df_original[input_cols+out_cols+temp_cols].copy()
        else:
          res = df_original[[case_col]+input_cols+out_cols+temp_cols].copy()
        
        global df
        df = res
        global output_selectors    
        global temporal_selectors
    
        output_selectors = create_output_columns_analysis(df)
        selector_box = HBox(output_selectors)
        res_box = VBox([Label('Choose values of Output Columns',layout=label_layout),selector_box])
        button_box = HBox([show_parameters])
        res_outputs = VBox([res_box,button_box])

        if temp_cols:
          temporal_selectors = create_tempo_columns_analysis(df)
          selector_box1 = HBox(temporal_selectors)
          res_box1 = VBox([Label('Choose reference Columns',layout=label_layout),selector_box1])
          res_all = VBox([res_box,res_box1])
          out.clear_output()
          with out:
              display(res_all)
              display(button_box)

        else:
           out.clear_output()
           with out:
              display(res_outputs)
        
        temporal_columns.value = ()
        


    show_data_button.on_click(on_button_show_click)
    
    def validation_of_input(df,
                            input_cols,
                            case_col,
                            temp_cols):
  
      if temp_cols:
        for i in temporal_selectors:
          if len( [x for x in i.value]) !=2:
            raise RuntimeError("Invalid selection of reference columns!," 
                              +" please select 2 reference variabels.")
        
        if(not all(df[temp_cols].apply(
                    lambda row_series : all( (pd.isna(x) 
                                              or int(x)==1
                                              or int(x)==0)
                                    for x in row_series),axis = 0))):
         
     
          
          raise RuntimeError("Invalid values in auxilary columns!")
        
        for i in temporal_selectors:
          ref_inputs = [x for x in i.value]
        
          if(not all(df[ref_inputs].apply(
                    lambda row_series : all((int(x) ==1
                                             or int(x) ==0) 
                                    for x in row_series),axis =0))):
          
              raise RuntimeError("Invalid values in reference columns!")
      
        if(not all(df[temp_cols].apply(
                    lambda row_series : any( (pd.isna(x) 
                                              )
                                    for x in row_series),axis = 0))):
         
    
        
          raise RuntimeError("Invalid values in auxilary columns!")

        if( not all(df[input_cols].apply(
                lambda row_series : all(isinstance(x,int)
                                for x in row_series),axis = 0))):
         
          raise RuntimeError("Invalid value in the input data!")
            
        for i in temporal_selectors:
          tmp =[int(df.columns.get_loc(x)) for x in i.value]

          positive_temporal_col_data = df[df.apply(
               lambda row_series: all(x ==1 
                                      or x==0 for x in row_series),axis =1)]
          if not positive_temporal_col_data.iloc[:,tmp].all(axis = None):
               raise RuntimeError("Incorrect values in auxiliary column!")
        
            
   
      else:
        #missing value check
        inputs = list(x  for x in df.columns if x != case_col)
        if(not all(df[inputs].apply(
                lambda row_series : all(isinstance(x,int)
                                for x in row_series),axis = 0))):
            raise RuntimeError("Invalid value in the input data!")
         
   
    # input_labels chec
    # constant check
        if (any(df[input_cols].apply(lambda row_series: True if
              len(row_series.unique()) ==1 else False,axis = 0))):
            raise RuntimeError("Please respecify your input data",\
                                       +" constants are not allowed!")      
      

    def on_set_parameters_click(b):
      #output
      output_map = dict()
      for i in output_selectors:
          output_map[i.description] = [x for x in i.value]
      global multi_value_output 
      multi_value_output = False
       
      global df
      for k in output_map.keys():
          if len(output_map[k])>1 or output_map[k] == [0]:
            multi_value_output = True
      global output_label_final
      output_label_final = []
      if multi_value_output:
        res = []
        for i in output_map.keys():
          res.append(str(i)+str(set(output_map[i])))
          output_label_final=list(x.replace(" ","") for x in res)

      else:
          output_label_final = out_cols
      

      #tempcols
      global temp_map
      if temp_cols:
        temp_map = []
        for i in temporal_selectors:
          tmp =','.join([str(df.columns.get_loc(x) +1) for x in i.value])
          tmp_str = str(i.description) + "{" + tmp + "}"
          temp_map.append(tmp_str)
      else:
        temp_map = None
      try:
        validation_of_input(df,input_cols,case_col,temp_cols)
        out.clear_output()
        with out:
          display(df)
      except RuntimeError as e:
        out.clear_output()
        with out:
          display("Invalid data: {0}".format(e))

                            
                            
    
    show_parameters.on_click(on_set_parameters_click)
    
    

#-----------preprocess_cotrols--------------------------

def create_preprocess_controls(df):
    
    # n_cut
    n_cut_slider = widgets.IntSlider(
        value = 1,
        min = 1,
        max = 10,
        step = 1,
        description = 'N-cut value:')
    #display(n_cut_slider)
    
    # inc_score1
    inc_score1_slider = widgets.FloatSlider(
        value = 0.5,
        min = 0,
        max = 1,
        step = 0.01,
        description = 'Inc score #1:')
    
     # inc_score2
    inc_score2_slider = widgets.FloatSlider(
        value = 0,
        min = 0,
        max = 1,
        step = 0.01,
        disabled = True,
        description = 'Inc score #2:')
    
     # U
    u_slider1 = widgets.Dropdown(
        options = ['0', '1'],
        value = '0',
        description = 'U value:',
        disabled = True,
    )  
   
     # rename_cols
    rename_checkbox=widgets.Checkbox(
         value = False,
         description = 'Use letters for inputs',
        )
    
     # inc_score2_on_off
    inc_score2_checkbox = widgets.Checkbox(
         value = False,
         description = 'Inc score #2',
    )
    
     # out1=widgets.Output()
    def on_inc2_button_click(v):
         new_value = v['new']
         u_slider1.disabled = not new_value
         inc_score2_slider.disabled = not new_value
    inc_score2_checkbox.observe(on_inc2_button_click, 'value')    
    
     # Preprocess button
    preprocess_button = widgets.Button(
         value = False,
         description = 'Preprocess data',
    )

     # Mining button
    button_mining = widgets.Checkbox(
        value = False,
        description = '',
        disabled = False,
        indent = False
    )

    run_data_mining = widgets.Button(
         value = False,
         description = 'Run data mining',
    )
   
    input_text = widgets.Text(value='1', disabled=True)      
   
    def on_mining_checkbox(b):
         input_text.disabled = not button_mining.value
    button_mining.observe(on_mining_checkbox,names='value')

     # Output
    out = widgets.Output()

    def on_run_mining(d):
         number = 0
         try:
            number = int(input_text.value)
         except ValueError:
            print('Input must be a number')
            return
         k = len(df.columns)-1
         if  number > k:
            print('Number of variabels must '+
                  'be smaller than{}'.format(k))
            return
         n_cut = n_cut_slider.value
         inc_score1 = inc_score1_slider.value 
         inc_score2 = (inc_score2_slider.value if
                       inc_score2_checkbox.value else None)
         U = int(u_slider1.value) if inc_score2_checkbox.value else None

       # print('case_col {}'.format(case_col))
         possible_inputs = [x for x in df.columns if 
                         (x not in out_cols and x!=case_col)]
         if (df[possible_inputs].shape[1] ==
             df[possible_inputs].select_dtypes(include=
                                               np.number).shape[1]):
        
            result = cora.data_mining(df,
                                 out_cols,
                                 number,
                                 case_col,
                                 temp_map,
                                 n_cut,
                                 inc_score1,
                                 inc_score2,
                                 U)
    
            out.clear_output(wait=True)
            with out:
              display(result.sort_values('Score',ascending=False))
         else:
            out.clear_output(wait=True)
            with out:
              display('A nonumeric column is present')
    
    run_data_mining.on_click(on_run_mining)

    def on_button_click(b): 
        global OptimizationContext
        global preprocessed_data
        n_cut = n_cut_slider.value
        inc_score1 = inc_score1_slider.value 
        inc_score2 = inc_score2_slider.value if\
                     inc_score2_checkbox.value else None
        U = int(u_slider1.value) if inc_score2_checkbox.value else None

         # inverse = invert_checkbox.value
        rename = rename_checkbox.value
        OptimizationContext = cora.OptimizationContext(data = df,
                                         output_labels = output_label_final,
                                         input_labels = input_cols,
                                         case_col = case_col,
                                         temporal_labels = temp_map,
                                         n_cut = n_cut,
                                         inc_score1 = inc_score1, 
                                         inc_score2 = inc_score2,
                                         U=U,
                                         rename_columns = rename)
        
        if OptimizationContext.preprocessing is False:
           OptimizationContext._preprocess_data()  
        
        col_new = []
        for x in out_cols:
            col_new.append("Out_"+str(x))
        out.clear_output()
        with out:
          display(OptimizationContext.preprocessed_data_raw.rename(
              columns = {k: v for k, v in (zip(out_cols,col_new))} ,
                                           inplace = False))                   

          if OptimizationContext.preprocessed_data_raw[out_cols].values.sum()==0:
            display('The function has only negative minterms,\
                     please check the truth table!')
          
        preprocessed_data = OptimizationContext.preprocessed_data
        use_precomputed = True
    
    preprocess_button.on_click(on_button_click)
    MiningBox = HBox([Label('Data Mining'),button_mining])    
    
    aa = HBox([MiningBox,input_text,run_data_mining])
    a = HBox([inc_score1_slider, n_cut_slider])
    b = HBox([inc_score2_checkbox,inc_score2_slider, u_slider1])
    c = HBox([ rename_checkbox, preprocess_button])
    display(aa)
    display(a)
    display(b)
    display(c)
    display(out)

#------------------------solution--------------------------

def create_implicants_buttons():


    pi_chart_button = widgets.Button(
        value = False,
        description = 'PI chart',
    )

    pi_details = widgets.Button(
        value = False,
        description = 'Details',
    )
  
 
     # Output
    out = widgets.Output()
    #out.clear_output(wait=True)
    display(out)
    
    descriptive_string = widgets.Button(
        value = False,
        description = "Solution"
    )

    cov_slider = widgets.FloatSlider(
        value = 0.5,
        min = 0,
        max = 1,
        step = 0.01,
        #disabled = len(out_cols)>1,
        description = 'Coverage')
  


    

    def on_desc_click(b):

      if len(out_cols) == 1:
        irr_sums = OptimizationContext.get_irredundant_sums()
        #if (irr_sums[0].get_descriptive_string(
         #   cov_slider.value) == 'Warning!'):
          #out.clear_output()
          #with out:
          #  clear_output(wait=True)
           # display("Warning...")
        #else:
          #out.clear_output()
        with out:
            clear_output(wait=True)
            for x in irr_sums:
                display(x.get_descriptive_string(cov_slider.value))
      else:
        irr_systems = OptimizationContext.get_irredundant_systems()
        if(irr_systems[0].get_descriptive_string(
            cov_slider.value) == 'Warning!'):
          #out.clear_output()
          with out:
            clear_output(wait=True)
            display("Warning...")
        else:
          #out.clear_output()
          s='\n'.join(x.get_descriptive_string(cov_slider.value) for x in irr_systems)
          with out:
            clear_output(wait=True)
            print(s)

      


     # pi chart 
    def on_pi_chart_click(b):

      pi_chart = OptimizationContext.prime_implicant_chart()
      with out:
        clear_output(wait=True)
        display(pi_chart)
   
    def on_pi_details(b):
      pi_details = OptimizationContext.pi_details()
      sol_details = OptimizationContext.system_details()
      with out:
          clear_output(wait=True)
          display(pi_details)
          display(sol_details)



    a = HBox([pi_chart_button,pi_details])
    b = HBox([descriptive_string,cov_slider])
     
    display(a)
    display(b)

    pi_chart_button.on_click(on_pi_chart_click)
    pi_details.on_click(on_pi_details)
    descriptive_string.on_click(on_desc_click)



The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


In [4]:
#@title Choose Data
uploaded = files.upload()
fname, data = uploaded.popitem()
if not any(fname.lower().endswith(x) for x in ['csv','xls','xslx']):
  print('Please enter an Excel or a csv file.')
  




  

Saving data_mv.csv to data_mv.csv


In [5]:
#@title Import Data
import_controls(fname)

Output()

In [6]:
#@title  Specify Inputs and Outputs
create_input_controls(df_original)


Output()

In [7]:
#@title Set Parameters

create_preprocess_controls(df)

Output()

In [8]:
#@title Compute Solution
%unload_ext google.colab.data_table
create_implicants_buttons()

Output()

In [ ]:
#@title Initialize Visualization

! [ ! -d "/content/logigram" ] && git clone https://github.com/ZuzanaSebb/Logigram.git logigram_repo
import sys
if '/content/logigram_repo' not in sys.path:
  sys.path.append('/content/logigram_repo')
!pip install -q SchemDraw==0.6.0

import logigram as log
import SchemDraw
import SchemDraw.logic as logic
import SchemDraw.elements as elm
import re
%matplotlib inline

%config InlineBackend.figure_format = 'svg'

In [ ]:
#@title Draw Logic Diagram

from ipywidgets import widgets, HBox
from IPython.display import clear_output
from google.colab import files
import re
from enum import Enum
input_text= widgets.Textarea()
output = widgets.Output()
download_svg = widgets.Button(description='Download as SVG')
download_svg.on_click(lambda x: files.download('scheme.svg'))
download_png = widgets.Button(description='Download as PNG')
download_png.on_click(lambda x: files.download('scheme.png'))
download_pdf = widgets.Button(description='Download as PDF')
download_pdf.on_click(lambda x: files.download('scheme.pdf'))

ok_button=widgets.Button(description='OK')
def ok_clicked(btn):
  input_value=input_text.value.split("\n")
  clear_output(wait=True)
  display(input_text)
  display(ok_button)
  f=log.draw_schem(input_value)
  log.save_figure(f,"scheme","svg")
  log.save_figure(f,"scheme","png")
  log.save_figure(f,"scheme","pdf")
  Download_options=HBox([download_png,download_svg,download_pdf])
  display(Download_options)


ok_button.on_click(ok_clicked)


display(input_text)
display(ok_button)
display(output)
 
